# EOCSI EASI training session 2: Learning to transform, visualise, analyse, and export data

This notebook will demonstrate how you can work with a specific area of interest from a geospatial data file. 

Once you have completed running all of the steps in the notebook, you can use it as a reference to start analysing your own area of interest. This notebook covers the following steps, many of which are boardly applicable to remote sensing analyses

1. Select an area of interest from a shapefile (or other geospatial data format)
2. Load relevant data for your area using its geometry, and mask the data to that area
3. Calculate the relevant index for your analysis and take the median to get a representative composite
4. Automatically identify a valid threshold to distinguish different landcover types in your area of interest
5. Calculate the total area covered by a given land cover type, and plot how it changes over time
6. Export your dataset as a raster to be used in other workflows

In this example, we'll be looking at urban expansion around Jakarta


## Set up

### Import required packages and functions

In [ ]:
import datacube
from datacube.utils.aws import configure_s3_access
from datacube.utils.geometry import Geometry
from datacube.utils import masking
from datacube.utils.cog import write_cog
from dea_tools.spatial import xr_rasterize
import geopandas as gpd
from odc.algo import xr_geomedian
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.filters import threshold_otsu

### Make a directory in the tutorials folder to store results in

In [ ]:
# make a results directory to store output

if not os.path.exists("results"):
    os.makedirs("results")

### Configure s3 access

This is necessary for Sentinel-2 ("s2_l2a") products, which we'll use in this example. It is also required for Landsat ("landsatN_c2l2_*") products

In [ ]:
configure_s3_access(aws_unsigned=False, requester_pays=True);

## Open and explore geospatial data

In this example, we will explore urban expansion around Jakarta. We have provided the "Jakarta_districts.shp" file in the data folder for you to use in this example. This shapefile contains 46 kecamatan (sub-districts) within the provinsi (province) of Dki Jakarta. This is a subset of the data available from https://data.humdata.org/dataset/cod-ab-idn 

### Load the data using geopandas

Geopandas is a useful python package for working with geospatial vector data. It allows you to read geospatial files in as tables, and provides useful functions for working with geometries. To learn more about geopandas, visit the [geopandas documentation](https://geopandas.org/en/stable/docs/user_guide.html)

In [ ]:
# Specify the path to the shapefile

filename = "data/jakarta_districts.shp"

In [ ]:
# read the shapefile and store the outputs in a variable called "geodataframe"
# the "explode" function is used to ensure multipolygons are split into multiple single polygons

geodataframe = gpd.read_file(filename).explode(ignore_index=False, index_parts=False)

### Exploring the datafile

Geopandas has two useful functions for getting an initial look at your spatial data. The first is "explore", which displays the polygons on a map. The second is "head", which allows you to look at the first five rows of your data. The first is useful of understanding the spatial distribution, the second is useful for understanding the attributes in the file and their values.

In [ ]:
# Display the polygons on a map
geodataframe.explore()

In [ ]:
# Display first 5 rows of the dataframe
geodataframe.head()

## Select areas of interest

In this example, we'll look at urban growth over three years, in two different districts. For this example, we've chosen two districts, one on the eastern side of the city (Cilinging) and one on the southern side of the city (Cilandak). These two districts can be selected by providing a list containing their names, and then selecting rows from our geodataframe that contain these names in the "ADM3_EN" column. The code to do this is provided below, and will return a new variable "areas_to_load" containing only these two districts.

In [ ]:
# Choose locations to load:
desired_locations = ["Cilincing", "Cipayung"]
selection_column = "ADM3_EN"

areas_to_load = geodataframe.loc[geodataframe[selection_column].isin(desired_locations), :].set_index(selection_column)

areas_to_load

## Decide which satellite data to load

### Date range

We will look at the area for a period of three years, from the beginning of 2017 to the end of 2019.

### Satellite product

For this time range, we can use Sentinel-2, which has a higher revisit time than Landsat, as well as higher spatial resolution.

### Satellite measurements

For this analysis, we'll be calculating the enhanced normalized difference impervious surfaces index (ENDISI), proposed in [this paper by Chen et al.](https://www.spiedigitallibrary.org/journals/journal-of-applied-remote-sensing/volume-13/issue-01/016502/Enhanced-normalized-difference-index-for-impervious-surface-area-estimation-at/10.1117/1.JRS.13.016502.full?SSO=1)

Like all normalised difference indicies, it has a range of `[-1, 1]`.
Note that `MNDWI`, `swir_ratio` and `alpha` are all part of the ENDISI calculation. The mean is calculated for each observation.

$$
\begin{aligned}
\text{MNDWI} = \frac{\text{GREEN} - \text{SWIR1}}{\text{GREEN} + \text{SWIR1}}
\end{aligned}
$$

$$
\begin{aligned}
\text{swir_ratio} = \frac{\text{SWIR1}}{\text{SWIR2}}
\end{aligned}
$$

$$
\begin{aligned}
\text{alpha} = \frac{2 * \text{mean(BLUE)}}{\text{mean(swir_ratio) + mean(MNDWI}^2)}
\end{aligned}
$$

$$
\begin{aligned}
\text{ENDISI} = \frac{\text{BLUE} - \text{alpha}*(\text{swir_ratio} + \text{MNDWI}^2)}
{\text{BLUE} + \text{alpha}*(\text{swir_ratio} + \text{MNDWI}^2)}
\end{aligned}
$$

Based on the equations above, the following measurements need to be loaded:
- `green`
- `blue`
- `swir_1`
- `swir_2`

We will also load the `mask` band to remove cloud affected pixels from Sentinel-2 imagery.

### Resolution and projection

We will load all pixels at 20m resolution, as the `swir` measurements are only available at this resolution.

We will load the data in its native coordinate reference system (CRS) to avoid transforming the data. The native crs for Sentinel-2 data over Jakarta is EPSG:32748. To determine this, we selected a Sentinel-2 tile in the [EASI Explorer](https://explorer.asia.easi-eo.solutions/products/s2_l2a/datasets/972e35d4-dfe3-5578-807d-384cdd3ebb0b) and viewed the metadata.

### Dask

The dask chunks setting allows our data to be processed in parallel. For the districts we're exploring, chunks of 500 pixels by 500 pixels is sufficient.

In [ ]:
# Date range
time_range = ("2017-01-01", "2019-12-31")

# Satellite product
products = ["s2_l2a"]

# Satellite measurements
measurements = ["green", "blue", "swir_1", "swir_2", "mask"]

# Resoultion and reprojection
resolution = (20, -20)
output_crs = "EPSG:32748"

# Dask
dask_chunks = {"time": 1, "x":500, "y":500}

In [ ]:
# Add the settings into a dictionary, which can be used for all areas of interest
query = {
    "time": time_range,
    "product": products,
    "measurements": measurements,
    "resolution": resolution,
    "output_crs": output_crs,
    "group_by": "solar_day",
    "dask_chunks": dask_chunks,
}

## Load data for each area of interest

The following code runs a number of steps for each area of interest in the `areas_to_load` geodataframe:

1. Add the polygon for the area to the datacube query dictionary
2. Load the data specified by the query dictionary
3. Remove cloud affected pixels using the Sentinel-2 mask band
4. Remove pixels outside the area of interest
5. Calculate ENDISI for all observations
6. Calculate the annual median ENDISI value for each pixel
7. Return the annual median ENDISI values

The results are stored in a dictionary called `output_results`, where the key is the name of each area of interest.

In [ ]:
# Load data for areas within the shapefile

from dask.diagnostics import ProgressBar

# Connect to the datacube
dc = datacube.Datacube()

# Create a python dictionary to store the results
output_results = {}

for index, area in areas_to_load.iterrows():
    
    # Get area name
    area_name = index
    
    # Get the polygon geometry for the event and add it to the query
    geometry = Geometry(geom=area.geometry, crs=areas_to_load.crs)
    query.update({"geopolygon": geometry})
    
    ds_s2 = dc.load(**query)
    
    # Apply cloud masking
    cloud_free_mask = (
        masking.make_mask(ds_s2.mask, qa="vegetation") | 
        masking.make_mask(ds_s2.mask, qa="bare soils") |
        masking.make_mask(ds_s2.mask, qa="water") |
        masking.make_mask(ds_s2.mask, qa="snow or ice")
    )
    
    ds_s2 = ds_s2.where(cloud_free_mask)
    
    ## Apply area masking
    area_mask = xr_rasterize(areas_to_load.loc[[index]], ds_s2)
    ds_s2 = ds_s2.where(area_mask)
    
    # Calculate ENDISI
    blue = ds_s2.blue
    mean_blue = blue.mean(dim=("x","y"))

    swir_ratio = ds_s2.swir_1/ds_s2.swir_2
    mean_swir_ratio = swir_ratio.mean(dim=("x","y"))

    mndwi = (ds_s2.green - ds_s2.swir_1)/(ds_s2.green + ds_s2.swir_1)
    mndwi_sq = mndwi**2
    mean_mndwi_sq = mndwi_sq.mean(dim=("x","y"))

    alpha = (2*mean_blue)/(mean_swir_ratio + mean_mndwi_sq)
    
    ds_s2["endisi"] = (blue - alpha*(swir_ratio + mndwi_sq))/(blue + alpha*(swir_ratio + mndwi_sq))
    
    # Calculate median ENDISI for each year to function as a mosaic
    print(f"Computing annual median ENDISI for {area_name}")
    with ProgressBar():
        annual_median_endisi = ds_s2["endisi"].groupby('time.year').median().compute()
    
    # Export results
    output_results[area_name] = annual_median_endisi


## Visualise data for each area of interest

It is useful to visualise our initial results to understand them in more detail. ENDISI has higher values for impervious surfaces, like roads and buildings, all found in urban areas. The plots below should give you an understanding of how urbanised each district is, based on the proportion of high ENDISI values compared to low ENDISI values.

For each area, we'll extract the data into a new variable to make it easier to work with

### Cipayung

In [ ]:
cipayung_endisi = output_results["Cipayung"]

cipayung_endisi.plot(col='year', col_wrap=3)

### Cilincing

In [ ]:
cilincing_endisi = output_results["Cilincing"]

cilincing_endisi.plot(col='year', col_wrap=3)

## Identifying urban areas through thresholding

While higher values of ENDISI indicate impervious surfaces common in urban areas, there is no universal cut-off value that distinguishes between urban and non-urban. To estimate such a value, it is useful to look at the histogram of ENDISI values for each area, with the expectation that there are two general classes we want to distinguish - urban and non-urban. 

The next cell defines a function that automatically estimates the threshold for separating the two classes, and displays this on a histogram. The method used is the Otsu Threshold method, implemented in [scikit-image](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html).

In [ ]:
# Function to calculate and display automatic threshold

def get_threshold(da):
    
    da_valid = da.values[~np.isnan(da.values)]
    threshold = threshold_otsu(da_valid)
    
    fig, ax = plt.subplots(figsize=(10, 4))
    da.plot.hist(
        ax=ax,
        bins=1000,
        range=(-1, 1),
        facecolor='gray',
    )
    ax.axvline(x=threshold)
    ax.set_title("Index Histogram")
    ax.set_ylabel("Pixel Count")
    ax.set_xlabel("Index Value")
    
    plt.show()
    
    return threshold

### Cipayung

In [ ]:
cipayung_threshold = get_threshold(cipayung_endisi)

print(f"The threshold for urban/non-urban in Cipayung is {round(cipayung_threshold, 2)}")

### Cilincing

In [ ]:
cilincing_threshold = get_threshold(cilincing_endisi)

print(f"The threshold for urban/non-urban in Cilincing is {round(cilincing_threshold, 2)}")

## Estimating change in urban area over time

Now that we have values to use for thresholding, it is possible to classify the ENDISI value as either urban (ENDISI values greater than the threshold) and non-urban (values lower than the threshold). From this, we can multiply the number of pixels classified as urban in a given year by the area covered by each pixel to estimate the total geographic area. Given we know the total area of each district, it is also possible to estimate the proportion of the district that is made up of urban structures.

In [ ]:
# Units for area estimation

area_per_pixel = abs(resolution[0])**2   # area per pixel in metres squared
msq_per_kmsq = 1000000   # number of square metres per square kilometre


In [ ]:
# Area units
areas_size = areas_to_load.to_crs(output_crs)
areas_size["area_sqkm"] = areas_size.area / msq_per_kmsq

areas_size

### Cipayung

In [ ]:
# Extract the total area of the district 
cipayung_total_area = areas_size.at["Cipayung", "area_sqkm"]

# Classify all pixels with ENDISI values above the threshold as urban. Returns 1 if urban, 0 if non-urban
cipayung_urban = (cipayung_endisi >= cipayung_threshold)

# count the number of urban pixels by summing up the values, then multiply by area per pixel and divide by the number of square metres in a square kilometre
cipayung_urban_area_kmsq = cipayung_urban.sum(dim=("x", "y")) * area_per_pixel / msq_per_kmsq

# get the proportion of the district classified as urban by dividing the estimate
cipayung_urban_proportion = cipayung_urban_area_kmsq / cipayung_total_area

In [ ]:
# Display the urban/non urban classification (urban in yellow, non urban in blue)
cipayung_urban.plot(col='year', col_wrap=3, add_colorbar=False)

### Cilincing

In [ ]:
# Extract the total area of the district 
cilincing_total_area = areas_size.at["Cilincing", "area_sqkm"]

# Classify all pixels with ENDISI values above the threshold as urban. Returns 1 if urban, 0 if non-urban
cilincing_urban = (cilincing_endisi >= cilincing_threshold)

# count the number of urban pixels by summing up the values, then multiply by area per pixel and divide by the number of square metres in a square kilometre
cilincing_urban_area_kmsq = cilincing_urban.sum(dim=("x", "y")) * area_per_pixel / msq_per_kmsq

# get the proportion of the district classified as urban by dividing the estimate
cilincing_urban_proportion = cilincing_urban_area_kmsq / cilincing_total_area

In [ ]:
cilincing_urban.plot(col='year', col_wrap=3, add_colorbar=False)

## Displaying change in total area and proportional area for both districts

The following code leverages Xarray's in-built plotting library, as well as matplotlib.

In [ ]:
year_ticks = [2017, 2018, 2019]
labels = []

fig, (ax_total, ax_prop) = plt.subplots(1, 2, figsize=(12, 4))

# Total Urban Area plot
cipayung_urban_area_kmsq.plot(ax=ax_total)
cilincing_urban_area_kmsq.plot(ax=ax_total)
ax_total.set_xticks(year_ticks)
ax_total.set_title("Total area classified as Urban over time")
ax_total.set_xlabel("Year")
ax_total.set_ylabel("Total urban area (km$^2$)")
ax_total.set_xticks(year_ticks)

# Proportion plot - multiply by 100 to get a percentage
(cipayung_urban_proportion * 100).plot(ax=ax_prop)
labels.append("Cipayung")
(cilincing_urban_proportion * 100).plot(ax=ax_prop)
labels.append("Cilincing")
ax_prop.set_title("Percentage area classified as Urban over time")
ax_prop.set_xlabel("Year")
ax_prop.set_ylabel("Percentage urban area")
ax_prop.set_xticks(year_ticks)

ax_prop.legend(labels, ncol=1, fontsize=12)

plt.show()

### Export the final figure

In [ ]:
figure_filename = "results/urban_area_change_jakarta.png"
fig.savefig(figure_filename, bbox_inches="tight", transparent=False)

## Export urban area rasters to geotiffs

This step is useful if you want to further analyse your data, or display it in another service. For each district, we'll save out a geotif of the urban areas, where a value of 1 indicates urban, and a value of 0 indicates non-urban.

In [ ]:
# Export cilincing urban product to geotiff
for i in range(len(cilincing_urban.year)):
    
    date = year_ticks[i]
    
    urban_area = cilincing_urban.isel(year=i).astype("uint8")
    
    write_cog(geo_im=urban_area,
              fname=f'results/cilincing_urban_extent_{date}.tif',
              overwrite=True)

In [ ]:
# Export cipayung urban product to geotiff
for i in range(len(cipayung_urban.year)):
    
    date = year_ticks[i]
    
    urban_area = cipayung_urban.isel(year=i).astype("uint8")
    
    write_cog(geo_im=urban_area,
              fname=f'results/cipayung_urban_extent_{date}.tif',
              overwrite=True)

## Practice now

Make a copy of this notebook so that you can keep this one as a reference. Some things you might like to try:

1. Replace the shapefile used in this example with one relevant to your use case, and adapt which areas you select for analysis.
2. Calculate a different band index when loading the data. Think about which measurments you'll need, and how to calculate the values.
3. Look online for how to make visually appealing plots with matplotlib, and see if you can customise the plots further.
